In [2]:
import csv
import pandas as pd
from tqdm import tqdm

In [3]:
NREL_PATH = '../data/NREL_raw.csv'

In [12]:
df = pd.read_csv(NREL_PATH, delimiter='\t', low_memory=False)
df[df['id'] == 1517]

,id,station_name,street_address,intersection_directions,city,state_name,zip,plus4,station_phone,access_days_time,...,access_code,access_detail_code,federal_agency_code,facility_type,ev_pricing,ev_on_site_renewable_source,restricted_access,nps_unit_name,maximum_vehicle_class,ev_workplace_charging
0,1517,LADWP - Truesdale Center,11797 Truesdale St,NaN,Sun Valley,CA,91352,NaN,NaN,Fleet use only,...,private,NaN,NaN,UTILITY,NaN,NaN,NaN,NaN,NaN,True


In [8]:
with open(NREL_PATH, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    columns = next(reader)
    while True:
        row = next(reader)
        if row[0] == '53413':
            break
    for i, item in enumerate(row):
        if item == '':
            row[i] = 'NULL'
        else:
            row[i] = row[i].replace('"', '\'')
    command = f'INSERT INTO nrel (' + ', '.join(columns) + \
        ') VALUES (' + ', '.join(row) + ')'
    print(command)

INSERT INTO nrel (id, station_name, street_address, intersection_directions, city, state_name, zip, plus4, station_phone, access_days_time, cards_accepted, ev_level1_evse_num, ev_level2_evse_num, ev_dc_fast_count, ev_other_info, ev_network, ev_network_web, geocode_status, latitude, longitude, date_last_confirmed, updated_at, owner_type_code, federal_agency_id, federal_agency_name, open_date, ev_connector_types, country, access_code, access_detail_code, federal_agency_code, facility_type, ev_pricing, ev_on_site_renewable_source, restricted_access, nps_unit_name, maximum_vehicle_class, ev_workplace_charging) VALUES (53413, Lewis & Clark Community College, 5800 Godfrey Rd, One station between Erickson Hall and the Nursing Building in the 'Cabin Lot.' The other station is between Riverbend Arena and the Math Building., Godfrey, IL, 62035, NULL, 618-468-2782, 24 hours daily; visit security desk after normal business hours to use, NULL, NULL, 2.0, NULL, NULL, Non-Networked, NULL, GPS, 38.950

In [83]:
def csv2sql_schema(filepath, table_name, delimiter=','):
    df = pd.read_csv(filepath, delimiter=delimiter)
    type_map = {
        'int64': 'INTEGER',
        'float64': 'FLOAT',
        'object': 'TEXT'
    }

    columns_sql = []
    columns_sql.append(f"  id INTEGER PRIMARY KEY AUTOINCREMENT,\n")
    for column, dtype in df.dtypes.items():
        if column == 'ID' or 'French' in column:
            continue
        column = column.lower().replace(' ', '_').replace('-', '_')
        if column == 'state':
            column = 'state_name'
        columns_sql.append(f"  {column} {type_map[dtype.name]},\n")
    
    command = f"CREATE TABLE {table_name} (\n{''.join(columns_sql)});"
    command = command[:-4] + '\n);'
    return command

In [86]:
txt = csv2sql_schema(NREL_PATH, 'nrel', '\t')

/tmp/ipykernel_10050/3190012416.py:2: DtypeWarning: Columns (6,20,31,46,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, delimiter=delimiter)


In [85]:
with open('../electrifind/schema.sql', 'a', encoding='utf-8') as f:
    f.write('\n\n')
    f.write(txt)
f.close()